In [38]:
import pandas as pd
import numpy as np
import pickle as pkl
import xgboost as xgb

In [59]:
results = pkl.load(open("../results/massive_trans/model_selection.pkl", "rb"))



In [60]:
for learner, learner_results in results.items():
    print(learner)
    
    for sec in ["if_pehe", "pseudo_outcome_r", "pseudo_outcome_dr"]:
        
        print(learner, sec, np.mean(learner_results[sec]))
    

XLearner
XLearner if_pehe 47.35904988946201
XLearner pseudo_outcome_r 0.40788540779731103
XLearner pseudo_outcome_dr 0.8505445472334389
DRLearner
DRLearner if_pehe 58.064512304633936
DRLearner pseudo_outcome_r 0.4285006485114753
DRLearner pseudo_outcome_dr 0.8883961034890184
SLearner
SLearner if_pehe 0.30767972263572363
SLearner pseudo_outcome_r 0.38649332306236117
SLearner pseudo_outcome_dr 0.7959641128172676
TLearner
TLearner if_pehe 36.251844804948675
TLearner pseudo_outcome_r 0.39720098677169635
TLearner pseudo_outcome_dr 0.8184201921795603
